In [2]:
import numpy as np
import pandas as pd
from func import *

np.set_printoptions(precision=3, suppress=True)

In [3]:
#inf = np.finfo(np.float32).max
inf = 1_000_000

In [4]:
C = np.array([
    [0, 0],
    [0, 4],
    [2, 2],
    [4, 4]
])
assert np.shape(C)[1] == 2, "Las coordenadas se componen de 2 valores."

In [5]:
S = np.array([
    [0,0,1],
    [0,0,1],
    [1,1,1],
    [1,1,1]
])
assert np.shape(S)[1] == 3, "La matriz S debe estar compuesta de 3 columnas."
assert np.shape(S)[0] == np.shape(C)[0], "La matriz S y la matriz C deben tener la misma forma."

In [6]:
B = np.array([
    [0,1],
    [0,2],
    [1,2],
    [1,3],
    [2,3]
])

In [7]:
EAI = np.array([
    [1, inf, 1],
    [1, inf, 1],
    [1, inf, 1],
    [1, inf, 1],
    [1, inf, 1]
])

In [8]:
L = np.linalg.norm(np.diff(C[B], axis=1), axis=2)
L

array([[4.   ],
       [2.828],
       [2.828],
       [4.   ],
       [2.828]])

In [9]:
with np.errstate(divide='ignore'):
    alpha = np.arctan(np.divide(*(np.diff(C[B], axis=1).squeeze().T[::-1])))
np.rad2deg(alpha)

array([ 90.,  45., -45.,   0.,  45.])

In [10]:
EAIL = np.hstack([EAI, L])

In [11]:
with np.errstate(divide='ignore', ):    
    Ki = []
    A = []
    Kt = []
    for el, t in zip(EAIL, alpha):
        Ki.append(stiffness_2d(*el))
        A.append(rot_2d(t))
        Kt.append(A[-1].T @ Ki[-1] @ A[-1])
    Ki = np.array(Ki)
    A = np.array(A)
    Kt = np.array(Kt)

In [12]:
def format(value) -> str:
    #if np.abs(value) > 10000:
    #    return 'inf'
    return value.format

pd.options.display.float_format = '{:.3F}'.format

In [13]:
Kg = np.zeros(shape=(S.size,S.size))
GLg = np.zeros(shape=(S.size, 1))
_idx = np.arange(0, S.size)
G_idx = _idx.reshape(len(S), -1)[B].reshape(-1, 6)
for idx, Ki, GL in zip(G_idx, Kt, S[B]):
    Kg[*np.meshgrid(idx, idx)] += Ki
    # GLg[idx] += GL.ravel()
GLg = S.ravel()
Kg.shape, GLg.shape

((12, 12), (12,))

In [14]:
cols = _idx.reshape(S.shape)[B].reshape(-1, 6)


In [15]:
show = pd.DataFrame(Kt[0])
show

,0,1,2,3,4,5
0,0.188,0.000,-0.375,-0.188,-0.000,-0.375
1,0.000,250000.000,0.000,-0.000,-250000.000,0.000
2,-0.375,0.000,1.000,0.375,-0.000,0.500
3,-0.188,-0.000,0.375,0.188,0.000,0.375
4,-0.000,-250000.000,-0.000,0.000,250000.000,-0.000
5,-0.375,0.000,0.500,0.375,-0.000,1.000


In [16]:
Kg.shape

(12, 12)

In [17]:
pd.DataFrame(Kg, index=_idx + 1, columns=_idx + 1)

,1,2,3,4,5,6,7,8,9,10,11,12
1,176777.148,176776.430,-0.905,-0.188,-0.000,-0.375,-176776.960,-176776.430,-0.530,0.000,0.000,0.000
2,176776.430,426776.960,0.530,-0.000,-250000.000,0.000,-176776.430,-176776.960,0.530,0.000,0.000,0.000
3,-0.905,0.530,2.414,0.375,-0.000,0.500,0.530,-0.530,0.707,0.000,0.000,0.000
4,-0.188,-0.000,0.375,426777.148,-176776.430,0.905,-176776.960,176776.430,0.530,-250000.000,0.000,0.000
5,-0.000,-250000.000,-0.000,-176776.430,426777.148,0.905,176776.430,-176776.960,0.530,0.000,-0.188,0.375
6,-0.375,0.000,0.500,0.905,0.905,3.414,-0.530,-0.530,0.707,0.000,-0.375,0.500
7,-176776.960,-176776.430,0.530,-176776.960,176776.430,-0.530,530330.881,176776.430,-0.530,-176776.960,-176776.430,-0.530
8,-176776.430,-176776.960,-0.530,176776.430,-176776.960,-0.530,176776.430,530330.881,-0.530,-176776.430,-176776.960,0.530
9,-0.530,0.530,0.707,0.530,0.530,0.707,-0.530,-0.530,4.243,0.530,-0.530,0.707
10,0.000,0.000,0.000,-250000.000,0.000,0.000,-176776.960,-176776.430,0.530,426776.960,176776.430,0.530


In [18]:
pd.DataFrame(np.diag(GLg) @ Kg @ np.diag(GLg), index=_idx + 1, columns=_idx + 1)\
    .pipe(lambda x: x.loc[(x != 0).any(axis=0), (x != 0).any(axis=1)])

,3,6,7,8,9,10,11,12
3,2.414,0.500,0.530,-0.530,0.707,0.000,0.000,0.000
6,0.500,3.414,-0.530,-0.530,0.707,0.000,-0.375,0.500
7,0.530,-0.530,530330.881,176776.430,-0.530,-176776.960,-176776.430,-0.530
8,-0.530,-0.530,176776.430,530330.881,-0.530,-176776.430,-176776.960,0.530
9,0.707,0.707,-0.530,-0.530,4.243,0.530,-0.530,0.707
10,0.000,0.000,-176776.960,-176776.430,0.530,426776.960,176776.430,0.530
11,0.000,-0.375,-176776.430,-176776.960,-0.530,176776.430,176777.148,-0.905
12,0.000,0.500,-0.530,0.530,0.707,0.530,-0.905,2.414
